In [20]:
import cv2
import os
import numpy as np
import matplotlib as plt
from skimage import io

In [2]:
video = cv2.VideoCapture('LipoKDEL1ug_biotin_no drug_live_5min1h.avi')
fps = video.get(cv2.CAP_PROP_FPS)
print('frames per second =',fps)

# seconds = 0.0857142857
# frame_id = int(fps*(60 + seconds))
# print('frame id =',frame_id)

frames per second = 7.0


In [3]:
seconds = 0.0857142857
frame_id = int(fps*(seconds))
print('frame id =',frame_id)

frame id = 0


In [4]:
video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
ret, frame = video.read()

In [ ]:
cv2.imshow('frame', frame); cv2.waitKey(0)
cv2.imwrite('LipoKDEL1ug_biotin_no drug_live_5min1h.png', frame)

In [25]:
#Definitely need to change this. Just used this to get over the initial hurdle
def extract_frames(video_path, frame_rate=12, output_dir="frames"):
  """Extracts frames from a video file and saves them with unique names.

  Args:
    video_path: The path to the video file.
    frame_rate: The desired frame rate for the extracted frames.
    output_dir: The directory to save the extracted frames.

  Returns:
    A list of extracted frames.
  """

  cap = cv2.VideoCapture(video_path)

  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  frame_interval = frame_count // frame_rate

  frames = []
  for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
      break
    if i % frame_interval == 0:
      frames.append(frame)

  # Create the output directory if it doesn't exist
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  # Save the extracted frames with unique names
  for i, frame in enumerate(frames, start = 1):
    filename = f"LipoKDEL1ug_biotin_no drug_live_5min1h_frame_{i:02d}.jpg"  # Use 2-digit padding for filenames
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, frame)

  return frames

# Extract frames from the video file
current_directory = os.getcwd()
video_path = os.path.join(current_directory, "LipoKDEL1ug_biotin_drug25uM_live_5min1h.avi")
frames = extract_frames(video_path)

print(f"Extracted {len(frames)-1} frames.")

Extracted 12 frames.


In [29]:
#Actual start
img_data = plt.imread('LipoKDEL1ug_biotin_no drug_live_5min1h_frame_01.jpg')
plt.imshow(img_data, cmap='jet')
plt.colorbar()

total_fluorescence = []

for i in range(1, 12+1):
    img = io.imread('LipoKDEL1ug_biotin_no drug_live_5min1h_frame_{i:02}.jpg')
    total_fluorescence.append(img.sum())

print(total_fluorescence)

AttributeError: module 'matplotlib' has no attribute 'imshow'